# Exploring Convolutional Layers Through Data and Experiments

## Context and Motivation
En este trabajo, exploraremos cómo las decisiones arquitectónicas en redes neuronales convolucionales (CNN) afectan el aprendizaje, usando un dataset real y experimentos controlados.

## Dataset Selection and Justification
Para este ejercicio, selecciono el dataset **Fashion-MNIST** porque:
- Es un conjunto de imágenes de ropa (28x28 píxeles, escala de grises, 10 clases).
- Es ampliamente usado para experimentos con CNNs.
- Su tamaño permite cargarlo en memoria fácilmente.
- Presenta suficiente complejidad para observar diferencias entre arquitecturas.

## 1. Dataset Exploration (EDA)
Analizaremos el tamaño, distribución de clases, dimensiones de las imágenes y mostraremos ejemplos por clase.

In [ ]:
# Importar librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist
import seaborn as sns

In [ ]:
# Cargar el dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(f'Tamaño de entrenamiento: {x_train.shape}, Tamaño de test: {x_test.shape}')

In [ ]:
# Distribución de clases
plt.figure(figsize=(8,4))
sns.countplot(x=y_train)
plt.title('Distribución de clases en el set de entrenamiento')
plt.xlabel('Clase')
plt.ylabel('Cantidad')
plt.show()

In [ ]:
# Mostrar ejemplos por clase
plt.figure(figsize=(10,5))
for i in range(10):
    idx = np.where(y_train == i)[0][0]
    plt.subplot(2,5,i+1)
    plt.imshow(x_train[idx], cmap='gray')
    plt.title(class_names[i])
    plt.axis('off')
plt.suptitle('Ejemplos de cada clase')
plt.show()

### Preprocesamiento
- Normalizaremos los valores de píxeles a [0,1].
- No es necesario redimensionar, ya que todas las imágenes son 28x28.

In [ ]:
# Normalización
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

## 2. Baseline Model (Non-Convolutional)
Implementaremos una red neuronal simple solo con capas densas (fully connected) para establecer un punto de referencia.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# One-hot encoding de las etiquetas
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

# Definir el modelo baseline
baseline_model = Sequential([
    Flatten(input_shape=(28,28)),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

baseline_model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

baseline_model.summary()

In [ ]:
# Entrenar el modelo baseline
history_baseline = baseline_model.fit(
    x_train, y_train_cat,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)

In [ ]:
# Evaluar el modelo baseline
loss, acc = baseline_model.evaluate(x_test, y_test_cat, verbose=0)
print(f"Test accuracy: {acc:.4f} | Test loss: {loss:.4f}")

In [ ]:
# Graficar el desempeño del modelo baseline
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history_baseline.history['accuracy'], label='Train')
plt.plot(history_baseline.history['val_accuracy'], label='Val')
plt.title('Accuracy')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history_baseline.history['loss'], label='Train')
plt.plot(history_baseline.history['val_loss'], label='Val')
plt.title('Loss')
plt.legend()
plt.show()

### Observed Limitations
- El modelo baseline no captura la estructura espacial de las imágenes.
- Puede tener menor capacidad de generalización en comparación con una CNN.
- Sirve como referencia para comparar mejoras con arquitecturas convolucionales.

## 3. Convolutional Architecture Design
Diseñaremos una CNN simple y justificaremos cada decisión arquitectónica.

### Arquitectura propuesta
- 2 capas convolucionales (filtros 3x3, 32 y 64 filtros respectivamente)
- Stride 1, padding 'same' para preservar dimensiones
- Activación ReLU
- MaxPooling después de cada convolución
- Capa densa final para clasificación

Justificación: Esta arquitectura es suficientemente simple para observar el efecto de las capas convolucionales, pero suficientemente expresiva para mejorar sobre el baseline.

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Sequential

# Preparar datos para CNN (agregar canal)
x_train_cnn = np.expand_dims(x_train, -1)
x_test_cnn = np.expand_dims(x_test, -1)

cnn_model = Sequential([
    Input(shape=(28,28,1)),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

cnn_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

cnn_model.summary()

In [ ]:
# Entrenar la CNN
history_cnn = cnn_model.fit(
    x_train_cnn, y_train_cat,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)

In [ ]:
# Evaluar la CNN
loss_cnn, acc_cnn = cnn_model.evaluate(x_test_cnn, y_test_cat, verbose=0)
print(f"Test accuracy: {acc_cnn:.4f} | Test loss: {loss_cnn:.4f}")

In [ ]:
# Graficar desempeño de la CNN
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history_cnn.history['accuracy'], label='Train')
plt.plot(history_cnn.history['val_accuracy'], label='Val')
plt.title('Accuracy')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history_cnn.history['loss'], label='Train')
plt.plot(history_cnn.history['val_loss'], label='Val')
plt.title('Loss')
plt.legend()
plt.show()

## 4. Controlled Experiments on the Convolutional Layer
Exploraremos el efecto del tamaño del kernel (3x3 vs 5x5) manteniendo todo lo demás fijo.

In [ ]:
# Modelo con kernel 5x5
cnn_model_5x5 = Sequential([
    Input(shape=(28,28,1)),
    Conv2D(32, (5,5), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Conv2D(64, (5,5), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(10, activation='softmax')
])

cnn_model_5x5.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

history_cnn_5x5 = cnn_model_5x5.fit(
    x_train_cnn, y_train_cat,
    epochs=10,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)

loss_5x5, acc_5x5 = cnn_model_5x5.evaluate(x_test_cnn, y_test_cat, verbose=0)
print(f"Test accuracy (5x5): {acc_5x5:.4f} | Test loss: {loss_5x5:.4f}")

In [ ]:
# Comparar desempeño de 3x3 vs 5x5
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history_cnn.history['val_accuracy'], label='3x3 kernel')
plt.plot(history_cnn_5x5.history['val_accuracy'], label='5x5 kernel')
plt.title('Validation Accuracy')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history_cnn.history['val_loss'], label='3x3 kernel')
plt.plot(history_cnn_5x5.history['val_loss'], label='5x5 kernel')
plt.title('Validation Loss')
plt.legend()
plt.show()

## 5. Interpretation and Architectural Reasoning
Responde en tus propias palabras (guía inicial):

- ¿Por qué las capas convolucionales superaron (o no) al baseline?
  - Explotan la estructura espacial e introducen compartición de pesos y localidad, reduciendo parámetros y mejorando generalización.

- ¿Qué sesgo inductivo introduce la convolución?
  - Localidad, compartición de pesos e invariancia a traslación.

- ¿Cuándo no usar convoluciones?
  - Datos tabulares sin relación posicional, secuencias con dependencias no locales (preferir transformadores/RNNs), grafos (preferir GNNs).

## 6. Deployment in SageMaker
Entrenaremos y desplegaremos el modelo en un endpoint de SageMaker usando el SDK de SageMaker. Necesitarás:
- Credenciales AWS configuradas.
- Un `role` de IAM con permisos para SageMaker y S3.
- Un bucket S3 en tu región.

Usaremos un script de entrenamiento (`train.py`) empaquetado por el SDK.